<a href="https://colab.research.google.com/github/heesukjang/W209_DataViz_Summer2023/blob/main/Unit_Assignments/Week_08_Assignment_BubbleChart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## W209 Data Visualization, Summer 2023
**Week 8 Assignment<br>
Heesuk Jang**

D3, Altair<br>
This weeks assignment is to download your Twitter archive and plot a hashtag network where each node is a hashtag and links are between hashtags that appear in the same tweet.

If you don't tweet, feel free to use the attached FileAttachment("duto_guerra_tweets.json") or a different personal data archive.

Tableau<br>
Using the same data described above, make a bubble chart with the **size given by the number of edges to each hashtag** and the **color given by the cluster id**. You will need to process this count and the cluster id external to Tableau.

If you don't have a tool in mind, start with Python (e.g., with the networkX library) or TabPy. For GUI only, Gephi is another possibility that should be no problem, but we haven't done this example specifically using Gephi.

In [67]:
!pip install nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
import altair as alt
import pandas as pd
import numpy as np
import networkx as nx
import nx_altair as nxa
import json

# gmail account
from google.colab import drive
drive.mount('/content/gdrive')

pd.options.display.float_format = '{:.0f}'.format
pd.__version__

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'1.5.3'

In [69]:
with open('/content/gdrive/MyDrive/W209_DataViz/duto_guerra_tweets.json') as f:
  file = f.read()
  data = json.loads(file)

# two hashtags appear together in a tweet
print(f"len(data) = {len(data)}\nhashtags: count_edges = {len(data[181]['entities']['hashtags'])}, {data[181]['entities']['hashtags']}\n")
data[181]

len(data) = 5872
hashtags: count_edges = 3, [{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]



{'retweeted': False,
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'entities': {'user_mentions': [{'name': 'RStudio',
    'screen_name': 'rstudio',
    'indices': ['18', '26'],
    'id_str': '235261861',
    'id': '235261861'}],
  'urls': [{'url': 'https://t.co/sN6CjOYIhH',
    'expanded_url': 'http://johnguerra.co/viz/influentials/RStudioConf2018/',
    'display_url': 'johnguerra.co/viz/influentia…',
    'indices': ['114', '137']}],
  'symbols': [],
  'media': [{'expanded_url': 'https://twitter.com/duto_guerra/status/959841361070567425/photo/1',
    'indices': ['160', '183'],
    'url': 'https://t.co/FzHtxIYfVP',
    'media_url': 'http://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'id_str': '959840986485657600',
    'id': '959840986485657600',
    'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/DVIKSrBWkAAJuEG.jpg',
    'sizes': {'medium': {'w': '600', 'h': '558', 'resize': 'fit'},
     'thumb': {'w': '150', 'h': '150', 're

In [80]:
lt = data[181]['entities']['hashtags']
print(lt)


for i in range lt:


[{'text': 'rstudioconf', 'indices': ['59', '71']}, {'text': 'DataViz', 'indices': ['138', '146']}, {'text': 'DataScience', 'indices': ['147', '159']}]


In [ ]:
[{'text': 'dataviz', 'indices': ['109', '117']}]
[{'text': 'UIST2020', 'indices': ['13', '22']}, {'text': 'AR', 'indices': ['101', '104']}]
[{'text': 'dataViz', 'indices': ['119', '127']}]
[{'text': 'deckgl', 'indices': ['82', '89']}, {'text': 'dataviz', 'indices': ['169', '177']}, {'text': 'WebDev', 'indices': ['178', '185']}]
[{'text': 'vegaLite', 'indices': ['107', '116']}, {'text': 'WebDev', 'indices': ['136', '143']}]

In [84]:
for i in range(len(data)-5750):
  if len(data[i]['entities']['hashtags']) > 0:
    print(i, data[i]['entities']['hashtags'])

4 [{'text': 'dataviz', 'indices': ['40', '48']}]
17 [{'text': 'dataviz', 'indices': ['225', '233']}]
18 [{'text': 'dataviz', 'indices': ['267', '275']}]
20 [{'text': 'dataviz', 'indices': ['230', '238']}]
21 [{'text': 'dataviz', 'indices': ['96', '104']}]
23 [{'text': 'd3js', 'indices': ['60', '65']}]
26 [{'text': 'CSS', 'indices': ['92', '96']}]
36 [{'text': 'clearviewai', 'indices': ['51', '63']}]
44 [{'text': 'dataviz', 'indices': ['109', '117']}]
47 [{'text': 'UIST2020', 'indices': ['13', '22']}, {'text': 'AR', 'indices': ['101', '104']}]
50 [{'text': 'dataViz', 'indices': ['119', '127']}]
52 [{'text': 'dataviz', 'indices': ['121', '129']}]
56 [{'text': 'ieeevis', 'indices': ['33', '41']}]
57 [{'text': 'dataviz', 'indices': ['79', '87']}]
58 [{'text': 'eurovis', 'indices': ['48', '56']}, {'text': 'dataviz', 'indices': ['272', '280']}]
63 [{'text': 'eurovis', 'indices': ['31', '39']}]
66 [{'text': 'dataviz', 'indices': ['77', '85']}]
70 [{'text': 'EuroVis2020', 'indices': ['74', '86

### itertools.combinations:
itertools.combinations() provides us with **all the possible tuples** a sequence or set of numbers or letters used in the iterator and the elements are assumed to be unique on the basis of their positions which are distinct for all elements.

In [113]:
import itertools
# for i in range(len(data)-5800):
#   if len(data[i]['entities']['hashtags']) == 0:
#     continue
#   elif

hashtag_set = set() # For nodes
hashtag_list = list() # For edges

for tweet_dict in data[:1000]:
    print(tweet_dict)
    hashtag_dict = tweet_dict["entities"]["hashtags"]
    if len(hashtag_dict) == 0: # No hashtags in this tweet
        continue

    hashtags = [hashtag["text"] for hashtag in hashtag_dict]
    print('hashtags: ', hashtags)
    print('hashtag_set.update(hashtags):  ',hashtag_set.update(hashtags))
    # print(hashtag_set.update(hashtags))
    hashtag_set.update(hashtags)
    # hashtag_set.update(hashtags)
    if len(hashtags) > 1: # Multilpe hashtags in a tweet
        hashtag_list.append(list(itertools.combinations(hashtags, r=2)))
        print('hashtag_list: ', hashtag_list,'\n')

print('\n========================== RETURN ==========================')
print('NODES, hashtag_set: ',hashtag_set)
print('EDGES, hashtag_list: ',hashtag_list)

# 'hashtags': [{'text': 'WebDev', 'indices': ['16', '23']}, {'text': 'Meteorjs', 'indices': ['96', '105']}, {'text': 'reactjs', 'indices': ['106', '114']}]
# three hashtags: ['WebDev','Meteorjs','reactjs']
# itertool.combinations: [('WebDev', 'Meteorjs'), ('WebDev', 'reactjs'), ('Meteorjs', 'reactjs')]

{'retweeted': False, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'entities': {'user_mentions': [{'name': 'D3.js', 'screen_name': 'd3js_org', 'indices': ['33', '42'], 'id_str': '4597853354', 'id': '4597853354'}, {'name': 'datascience@berkeley', 'screen_name': 'BerkeleyData', 'indices': ['57', '70'], 'id_str': '1227698863', 'id': '1227698863'}], 'urls': [{'url': 'https://t.co/kB06XJRKZO', 'expanded_url': 'https://observablehq.com/@john-guerra/mids-curriculum-analysis', 'display_url': 'observablehq.com/@john-guerra/m…', 'indices': ['246', '269']}], 'symbols': [], 'media': [{'expanded_url': 'https://twitter.com/duto_guerra/status/1302979067080302593/photo/1', 'indices': ['270', '293'], 'url': 'https://t.co/Ek8QjDyy3p', 'media_url': 'http://pbs.twimg.com/tweet_video_thumb/EhUcfYwXcAAKQDt.jpg', 'id_str': '1302978992740462592', 'id': '1302978992740462592', 'media_url_https': 'https://pbs.twimg.com/tweet_video_thumb/EhUcfYwXcAAKQDt.jpg', 'sizes': {'thum

In [116]:
node_ls = []
for i in range(len(data)):
  # print(i, d[i]['entities']['hashtags'])
  for j in range(len(data[i]['entities']['hashtags'])):
    if len(data[i]['entities']['hashtags']) > 0:
      node_ls.append(data[i]['entities']['hashtags'][j]['text'].lower())
node_ls[:10]

['dataviz',
 'dataviz',
 'dataviz',
 'dataviz',
 'dataviz',
 'd3js',
 'css',
 'clearviewai',
 'dataviz',
 'uist2020']

In [130]:
import collections
count = collections.Counter(node_ls[:20])
dict(count)

# defaultdict(int,
#             {'dataviz': 11,
#              'd3js': 1,
#              'css': 1,
#              'clearviewai': 1,
#              'uist2020': 1,
#              'ar': 1,
#              'ieeevis': 1,
#              'eurovis': 2,
#              'eurovis2020': 1})

{'dataviz': 11,
 'd3js': 1,
 'css': 1,
 'clearviewai': 1,
 'uist2020': 1,
 'ar': 1,
 'ieeevis': 1,
 'eurovis': 2,
 'eurovis2020': 1}

In [126]:
from collections import defaultdict
node_dic = defaultdict(int)
for node in node_ls[:20]:
    print('node: ', node)
    node_dic[node] += 1
    print('node_dic[node]: ', node_dic[node], '\n')
node_dic

node:  dataviz
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  2 

node:  dataviz
node_dic[node]:  3 

node:  dataviz
node_dic[node]:  4 

node:  dataviz
node_dic[node]:  5 

node:  d3js
node_dic[node]:  1 

node:  css
node_dic[node]:  1 

node:  clearviewai
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  6 

node:  uist2020
node_dic[node]:  1 

node:  ar
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  7 

node:  dataviz
node_dic[node]:  8 

node:  ieeevis
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  9 

node:  eurovis
node_dic[node]:  1 

node:  dataviz
node_dic[node]:  10 

node:  eurovis
node_dic[node]:  2 

node:  dataviz
node_dic[node]:  11 

node:  eurovis2020
node_dic[node]:  1 



defaultdict(int,
            {'dataviz': 11,
             'd3js': 1,
             'css': 1,
             'clearviewai': 1,
             'uist2020': 1,
             'ar': 1,
             'ieeevis': 1,
             'eurovis': 2,
             'eurovis2020': 1})

In [85]:
[hashtag['text'] for hashtag in data[21]['entities']['hashtags']]

['dataviz']

In [110]:
# question: do we need to consider different letter case hashtags as two different hashtags? i.e. css vs CSS
nodes_ht = []
for i in range(len(data)-5810):
  if len(data[i]['entities']['hashtags']) > 0:
    ht = [hashtag['text'] for hashtag in data[i]['entities']['hashtags']]
    nodes_ht.extend(ht)
set(nodes_ht)

{'AR',
 'CSS',
 'UIST2020',
 'clearviewai',
 'd3js',
 'dataViz',
 'dataviz',
 'eurovis',
 'ieeevis'}